In [2]:
pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 1.9 MB/s eta 0:00:0000:0100:010m
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
# os.chdir(os.path.dirname(os.path.realpath(__file__)))
import sys
# sys.path.append('../../')
sys.path.append('../')
sys.path.append('../model_zoo/SimpleX/src/')
from matchbox import datasets
from datetime import datetime
from matchbox.utils import load_config, set_logger, print_to_json, print_to_list
from matchbox.features import FeatureMap, FeatureEncoder
# import model_zoo.SimpleX.src as models
from SimpleX import SimpleX
from matchbox.pytorch.torch_utils import seed_everything
import gc
import argparse
import logging
import os
from pathlib import Path
import torch

In [2]:
!ls ../model_zoo/SimpleX/config/

SimpleX_amazonbooks_m1 SimpleX_gowalla_m1     SimpleX_yelp18_m1


In [3]:
experiment_id = 'SimpleX_yelp18_m1'
params = load_config('../model_zoo/SimpleX/config/SimpleX_yelp18_m1', experiment_id)
params['gpu'] = -1

seed_everything(seed=params['seed'])
dataset = params['dataset_id'].split('_')[0].lower()
params['data_root'] = '../data/Yelp18/'
data_dir = os.path.join(params['data_root'], params['dataset_id'])

../matchbox/utils.py:33: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config_dict = yaml.load(cfg)
../matchbox/utils.py:53: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config_dict = yaml.load(cfg)


In [4]:
feature_encoder = FeatureEncoder(**params)
feature_encoder.feature_map.load(feature_encoder.json_file)
feature_map = feature_encoder.feature_map
params["train_data"] = os.path.join(data_dir, 'train*.h5')
params["valid_data"] = os.path.join(data_dir, 'valid*.h5')
if "test_data" in params:
    params["test_data"] = os.path.join(data_dir, 'test*.h5')
params["item_corpus"] = os.path.join(data_dir, 'item_corpus.h5')

In [5]:
model = SimpleX(feature_map, **params)
model.count_parameters() # print number of parameters used in model

In [6]:
user1 = [ 15206, 23624,  6230,  2522,  5402, 13547,   883,  1179,  3544,   549, 35264, 14587, 35951]
user2 = [ 3002,  8692, 48, 908,  2002,   478,  9388,  8423, 37628,    92,  4877,   920, 33681]
user_dict = {
    'user_history':torch.tensor([user1, user2]),
    'user_id':torch.tensor([27736,  7459])
}
item_dict = {
    'item_id': torch.tensor(user1 + user2)
}

In [7]:
# model.load_weights(model.checkpoint)

In [12]:
pred = model([user_dict, item_dict, []], training=False)['y_pred']

In [13]:
pred

tensor([[ 0.3096,  0.0758,  0.0694, -0.0739, -0.1653, -0.2259,  0.0904, -0.0189,
         -0.2329, -0.0122,  0.0015,  0.1261,  0.1827, -0.0286,  0.0899, -0.1308,
          0.0062, -0.1097, -0.0316, -0.0330,  0.1335, -0.3738,  0.1013,  0.1242,
         -0.1633,  0.1054],
        [ 0.0463,  0.0116,  0.0713, -0.0337,  0.1111, -0.0601,  0.0392,  0.0676,
         -0.0845,  0.0533, -0.0791,  0.0660, -0.0670, -0.1479, -0.0404, -0.2388,
         -0.0265, -0.0814,  0.0893,  0.2623,  0.1442,  0.0444,  0.0486, -0.2136,
          0.0819,  0.0058]], grad_fn=<SqueezeBackward1>)

In [28]:
# train_gen, valid_gen = datasets.h5_generator(feature_map, stage='train', **params)
# model.fit(train_gen, valid_generator=valid_gen, **params)


  0%|                                                                                                                                                                                                          | 0/2417 [00:26<?, ?it/s]


KeyboardInterrupt: 

In [ ]:


logging.info('****** Validation evaluation ******')
valid_result = model.evaluate(train_gen, valid_gen)
del valid_gen
gc.collect()

test_result = dict()
if "test_data" in params:
    logging.info('******** Test evaluation ********')
    test_gen = datasets.h5_generator(feature_map, stage='test', **params)
    test_result = model.evaluate(train_gen, test_gen)

with open(Path(args['config']).stem + '.csv', 'a+') as fw:
    fw.write(' {},[command] python {},[exp_id] {},[dataset_id] {},[train] {},[val] {},[test] {}\n' \
        .format(datetime.now().strftime('%Y%m%d-%H%M%S'), 
                ' '.join(sys.argv), experiment_id, params['dataset_id'],
                "N.A.", print_to_list(valid_result), print_to_list(test_result)))

